In [1]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

In [2]:
!pip show langchain

Name: langchain
Version: 0.2.3
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [3]:
!wget -q https://www.dropbox.com/s/zoj9rnm7oyeaivb/new_papers.zip
!unzip -q new_papers.zip -d new_papers

replace new_papers/new_papers/toolformer.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/__MACOSX/new_papers/._toolformer.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/new_papers/Flash-attention.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/__MACOSX/new_papers/._Flash-attention.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/new_papers/Augmenting LLMs Survey.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/__MACOSX/new_papers/._Augmenting LLMs Survey.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/new_papers/ReACT.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/__MACOSX/new_papers/._ReACT.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/new_papers/ALiBi.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace new_papers/__MACOSX/new_papers/._ALiBi.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - PDFs
- ChromaDB - with more meta data?
- Source info
- gpt-3.5-turbo API
- HuggingFace Embeddings
- Instuctor Embeddings


## Setting up LangChain


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

In [5]:
pip install -U langchain-community

In [6]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Load multiple and process documents

In [7]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [8]:
len(documents)

142

In [9]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [10]:
len(texts)

659

In [11]:
texts[3]

Document(page_content='has time and memory complexity quadratic in sequence length. An important question is whether making\nattention faster and more memory-eﬃcient can help Transformer models address their runtime and memory\nchallenges for long sequences.\nMany approximate attention methods have aimed to reduce the compute and memory requirements of\nattention. These methods range from sparse-approximation [ 51,74] to low-rank approximation [ 12,50,84],\nand their combinations [ 3,9,92]. Although these methods reduce the compute requirements to linear or\nnear-linear in sequence length, many of them do not display wall-clock speedup against standard attention\nand have not gained wide adoption. One main reason is that they focus on FLOP reduction (which may not\ncorrelate with wall-clock speed) and tend to ignore overheads from memory access (IO).\nIn this paper, we argue that a missing principle is making attention algorithms IO-aware [1]—that is,', metadata={'source': 'new_papers/

## HF Embeddings

In [12]:
# from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"

# hf = HuggingFaceEmbeddings(model_name=model_name)

## HF Instructor Embeddings

In [13]:
pip install sentence-transformers==2.2.2


In [14]:
pip show sentence_transformers

Name: sentence-transformers
Version: 2.2.2
Summary: Multilingual text embeddings
Home-page: https://github.com/UKPLab/sentence-transformers
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, nltk, numpy, scikit-learn, scipy, sentencepiece, torch, torchvision, tqdm, transformers
Required-by: 


In [15]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl",
    model_kwargs={"device": "cuda"}
)

load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [17]:
# persiste the db to disk
vectordb.persist()
vectordb = None

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [18]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [19]:
retriever = vectordb.as_retriever()

In [20]:
docs = retriever.get_relevant_documents("What is Flash attention?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [21]:
len(docs)

4

In [22]:
docs[0]

Document(page_content='access.\nWe propose FlashAttention , a new attention algorithm that computes exact attention with far fewer\nmemory accesses. Our main goal is to avoid reading and writing the attention matrix to and from HBM.\nThis requires (i) computing the softmax reduction without access to the whole input (ii) not storing the large\nintermediate attention matrix for the backward pass. We apply two well-established techniques to address\nthese challenges. (i) We restructure the attention computation to split the input into blocks and make several\npasses over input blocks, thus incrementally performing the softmax reduction (also known as tiling). (ii) We\nstore the softmax normalization factor from the forward pass to quickly recompute attention on-chip in the\nbackward pass, which is faster than the standard approach of reading the intermediate attention matrix from\nHBM. We implement FlashAttention in CUDA to achieve ﬁne-grained control over memory access and', metadata={'

In [23]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [24]:
retriever.search_type

'similarity'

In [25]:
retriever.search_kwargs

{'k': 3}

## Make a chain

In [26]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [27]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [28]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 Flash attention is a new attention algorithm that aims to reduce the number of memory accesses and improve
the quality and capabilities of Transformer models by using techniques such as tiling and storing softmax
normalization factors for faster computation. It is also faster and more memory-efficient than existing
attention methods.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [29]:
# break it down
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

 IO-aware refers to an approach or method that takes into account the input/output (IO) operations and
optimizes them for efficient memory usage and data transfer. In the context of deep learning, this would mean
developing algorithms or implementations that are aware of the IO operations and aim to reduce their impact on
memory and performance.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [30]:
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



Tiling in flash-attention is a technique used to split the input into blocks and make several passes over
input blocks, thus incrementally performing the softmax reduction. This helps to reduce the amount of memory
accesses required, making the algorithm more efficient.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [31]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that learns to use external tools via simple APIs in a self-supervised way.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [32]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



Toolformer can be used with tools such as search engines, calculators, and translation systems through simple
API calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [33]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 It is unclear how many examples are needed for each tool, as it may vary depending on the model and the task.
However, few-shot prompting only requires a handful of manually labeled examples.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [34]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The best retrieval augmentations for LLMs are dense neural retrievers and external tools, as they have been
shown to improve reasoning abilities and reduce the amount of non-factual information produced by LMs.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [35]:
query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 REALM (Guu et al. ,2020) was the first method to jointly train end-to-end a retrieval system with an encoder
LM, while RAG (Lewis et al. ,2020) jointly fine-tunes the retriever with a sequence-to-sequence model.
Additionally, Izacard and Grave (2020) introduced a modification of the seq2seq architecture to efficiently
process many retrieved documents.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/ReACT.pdf


In [36]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity',
 <langchain_community.vectorstores.chroma.Chroma at 0x7c33a17b83a0>)

In [37]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [38]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/3897c0c3-2c97-42c9-b464-f78e87c18853/ (stored 0%)
  adding: db/3897c0c3-2c97-42c9-b464-f78e87c18853/length.bin (deflated 69%)
  adding: db/3897c0c3-2c97-42c9-b464-f78e87c18853/link_lists.bin (stored 0%)
  adding: db/3897c0c3-2c97-42c9-b464-f78e87c18853/header.bin (deflated 61%)
  adding: db/3897c0c3-2c97-42c9-b464-f78e87c18853/data_level0.bin (deflated 100%)
  adding: db/chroma.sqlite3 (deflated 44%)


In [39]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [2]:

!unzip db.zip

y
Archive:  db.zip
replace db/3897c0c3-2c97-42c9-b464-f78e87c18853/length.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: db/3897c0c3-2c97-42c9-b464-f78e87c18853/length.bin  
replace db/3897c0c3-2c97-42c9-b464-f78e87c18853/link_lists.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: db/3897c0c3-2c97-42c9-b464-f78e87c18853/link_lists.bin  
replace db/3897c0c3-2c97-42c9-b464-f78e87c18853/header.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: db/3897c0c3-2c97-42c9-b464-f78e87c18853/header.bin  
replace db/3897c0c3-2c97-42c9-b464-f78e87c18853/data_level0.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: db/3897c0c3-2c97-42c9-b464-f78e87c18853/data_level0.bin  
replace db/chroma.sqlite3? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: db/chroma.sqlite3       


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [13]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [14]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [7]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [15]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [17]:
# full example
# query = "How much money did Pando raise?"
# llm_response = qa_chain(query)
# process_llm_response(llm_response)

### Chat prompts

In [11]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [12]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
